In [4]:
SiO2_unit = 300

N_Si = SiO2_unit
N_O = SiO2_unit * 2

density=2.2

import numpy as np

# Parameters
# 3種類の原子の数
n_atoms = [N_O, N_Si]
symbols = []

for i in range(N_O):
    symbols.append('O')

for i in range(N_Si):    
    symbols.append('Si')

from ase import Atom 
from ase import Atoms 

trial=Atoms(symbols=symbols)
masses=np.sum(trial.get_masses())

AngTocm=1.0e-8
cmToAng=1.0e8
Avogadro=6.022e23

volume=masses/Avogadro/density*cmToAng*cmToAng*cmToAng
lattice_a=np.power(volume,1.0/3.0)

# セルのサイズ
cell_size = [lattice_a, lattice_a, lattice_a]

print(cell_size)

# 原子間の最小距離
min_dist = 1.9

# Determine the number of cells per dimension
n_cells = [int(size / min_dist) for size in cell_size]

# Initialize the cell list
cell_list = [[[[] for _ in range(n_cells[2])] for _ in range(n_cells[1])] for _ in range(n_cells[0])]

# Periodic boundary condition aware distance calculation
def pbc_dist(atom1, atom2, cell_size):
    dist_vec = [abs(atom1[i] - atom2[i]) for i in range(len(atom1))]
    for i, vec in enumerate(dist_vec):
        if vec > cell_size[i] / 2.0:
            dist_vec[i] = cell_size[i] - vec
    return np.sqrt(sum([i**2 for i in dist_vec]))

# Initialize the atom list
atoms = []

initial=Atoms(cell=cell_size, pbc=True)

counter=0
for n in n_atoms:
    for _ in range(n):
        while True:
            # Generate a random position
            pos = [np.random.uniform(0, size) for size in cell_size]

            # Determine the cell of the current atom
            cell = [int(pos[i] / cell_size[i] * n_cells[i]) for i in range(3)]

            # Check the distance with the other atoms in the same cell and the neighboring cells
            close_cells = [((cell[0] + dx) % n_cells[0], (cell[1] + dy) % n_cells[1], (cell[2] + dz) % n_cells[2]) 
                           for dx in [-1, 0, 1] for dy in [-1, 0, 1] for dz in [-1, 0, 1]]

            close_atoms = [atom for c in close_cells for atom in cell_list[c[0]][c[1]][c[2]]]

            dists = [pbc_dist(pos, atom, cell_size) for atom in close_atoms]
            
            # If the minimum distance is larger than the threshold, accept the position
            if len(dists) == 0 or min(dists) > min_dist:
                atoms.append(pos)
                
                atom=Atom(symbols[counter],pos)
                initial.append(atom)
                cell_list[cell[0]][cell[1]][cell[2]].append(pos)
                
                counter+=1
                break


# %%
masses=np.sum(initial.get_masses())
#print(masses)

AngTocm=1.0e-8
cmToAng=1.0e8
Avogadro=6.022e23

volume=initial.get_volume()
density=masses/Avogadro/volume*cmToAng*cmToAng*cmToAng

print(density)

# %%
from ase.io import write
write('/home/nozawa/SiO2-Na2O_PaiNN/lammps/POSCAR_NS33', initial,format='vasp',vasp5=True, sort=True)

# %%
from ase.io import read, write 
atoms=read('/home/nozawa/SiO2-Na2O_PaiNN/lammps/POSCAR_NS33',format='vasp')

write('/home/nozawa/SiO2-Na2O_PaiNN/lammps/sample_NS33.data',atoms,format='lammps-data')



: 